<a href="https://colab.research.google.com/github/UN-GCPDS/Unemployment-Rate-Prediction/blob/main/Supervised_Approach_Generate_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Funciones y librerías

In [ ]:
# ========================================
# 1. Instalación de librerías externas
# ========================================
!pip install pmdarima==1.8.5
!pip install -q -U keras-tuner

# ========================================
# 2. Librerías base y utilidades generales
# ========================================
import warnings
import numpy as np
import pandas as pd
from tabulate import tabulate

# ========================================
# 3. Visualización
# ========================================
import matplotlib.pyplot as plt
import seaborn as sns

# ========================================
# 4. Preprocesamiento de datos
# ========================================
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline

# ========================================
# 5. Modelos de regresión clásicos
# ========================================
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, SGDRegressor, BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge

# ========================================
# 6. Modelos bayesianos y de procesos gaussianos
# ========================================
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel, Matern, ExpSineSquared, DotProduct

# ========================================
# 7. Modelos estadísticos y series temporales
# ========================================
import pmdarima as pm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tools.sm_exceptions import ConvergenceWarning

# ========================================
# 8. Evaluación de modelos
# ========================================
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, r2_score

# ========================================
# 9. Keras Tuner para ajuste de hiperparámetros
# ========================================
from keras_tuner import BayesianOptimization as OriginalBayesianOptimization
from keras_tuner.oracles import BayesianOptimizationOracle as OriginalBayesianOptimizationOracle
from keras_tuner import Tuner, Objective
from keras_tuner.tuners import SklearnTuner
from keras_tuner.src.engine import hyperparameters as hp_module
from keras_tuner.src.engine import oracle as oracle_module
from keras_tuner.src.engine import trial as trial_module
from keras_tuner.src.engine import tuner as tuner_module
from keras_tuner.src.api_export import keras_tuner_export

# ========================================
# 10. Configuración de warnings
# ========================================
warnings.filterwarnings("ignore", category=UserWarning, message="Non-stationary starting autoregressive parameters found")
warnings.filterwarnings("ignore", category=UserWarning, message="Non-invertible starting MA parameters found")
warnings.filterwarnings("ignore", category=ConvergenceWarning, message="Maximum Likelihood optimization failed to converge")

class CustomBayesianOptimizationOracle(OriginalBayesianOptimizationOracle):
    def __init__(self, *args, nu=0.5, **kwargs):
        self.nu = nu  # Establecer nu antes de llamar al constructor de la clase base
        super().__init__(*args, **kwargs)
        self.gpr = self._make_gpr()  # Asegurarse de que _make_gpr se llame después de establecer nu
    def _gpr_trained(self):
        if self.gpr is None:
            print("There is no GPR model trained yet.")
            return None
        else:
            return self.gpr
    def _make_gpr(self):
        return sklearn.gaussian_process.GaussianProcessRegressor(
            kernel=sklearn.gaussian_process.kernels.Matern(nu=self.nu),
            n_restarts_optimizer=20,
            normalize_y=True,
            alpha=self.alpha,
            random_state=self.seed,
        )

class CustomBayesianOptimization(Tuner):
    def __init__(
        self,
        hypermodel=None,
        objective=None,
        max_trials=10,
        num_initial_points=None,
        alpha=1e-4,
        beta=2.6,
        seed=None,
        hyperparameters=None,
        tune_new_entries=True,
        allow_new_entries=True,
        max_retries_per_trial=0,
        max_consecutive_failed_trials=3,
        nu=0.5,  # Valor por defecto para nu
        **kwargs
    ):
        oracle = CustomBayesianOptimizationOracle(
            objective=objective,
            max_trials=max_trials,
            num_initial_points=num_initial_points,
            alpha=alpha,
            beta=beta,
            seed=seed,
            hyperparameters=hyperparameters,
            tune_new_entries=tune_new_entries,
            allow_new_entries=allow_new_entries,
            max_retries_per_trial=max_retries_per_trial,
            max_consecutive_failed_trials=max_consecutive_failed_trials,
            nu=nu  # Pasar el parámetro nu
        )
        # Llama directamente al constructor de la clase base Tuner
        super().__init__(oracle=oracle, hypermodel=hypermodel, **kwargs)

class CustomSklearnBayesianOptimization(SklearnTuner):
    def __init__(
        self,
        hypermodel=None,
        objective=None,
        max_trials=10,
        num_initial_points=None,
        alpha=1e-1,
        beta=2.6,
        seed=None,
        hyperparameters=None,
        tune_new_entries=True,
        allow_new_entries=True,
        max_retries_per_trial=0,
        max_consecutive_failed_trials=3,
        nu=0.5,  # Valor por defecto para nu
        scoring=None,
        cv=None,
        **kwargs
    ):
        oracle = CustomBayesianOptimizationOracle(
            objective=objective,
            max_trials=max_trials,
            num_initial_points=num_initial_points,
            alpha=alpha,
            beta=beta,
            seed=seed,
            hyperparameters=hyperparameters,
            tune_new_entries=tune_new_entries,
            allow_new_entries=allow_new_entries,
            max_retries_per_trial=max_retries_per_trial,
            max_consecutive_failed_trials=max_consecutive_failed_trials,
            nu=nu  # Pasar el parámetro nu
        )
        # Llama directamente al constructor de la clase base SklearnTuner
        super().__init__(oracle=oracle, hypermodel=hypermodel, scoring=scoring, cv=cv, **kwargs)
class TimeSeriesPipeline:
    def __init__(self, model_type='SARIMA', seasonal=True, scaler=None, m=12):
        self.model_type = model_type
        self.scaler = scaler
        self.model = None
        self.seasonal = seasonal
        self.m = m  # Número de períodos en una temporada para SARIMA
    def fit(self, y_train):
        # Aplicar escalado si se proporciona
        if self.scaler:
            y_train = self.scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
        # Utilizar auto_arima para encontrar los mejores parámetros
        model_auto = auto_arima(
            y_train,
            seasonal=self.seasonal,
            m=self.m,
            trace=False,
            error_action='ignore',
            suppress_warnings=True,
            stepwise=True
        )
        try:
            order = model_auto.order
        except:
            model_auto = auto_arima(
            y_train,
            seasonal=self.seasonal,
            m=1,
            trace=False,
            error_action='ignore',
            suppress_warnings=True,
            stepwise=True)
        order = model_auto.order
        seasonal_order = model_auto.seasonal_order if self.seasonal else (0, 0, 0, 0)
        # Ajustar el modelo basado en el tipo especificado
        if self.model_type == 'ARIMA':
            self.model = ARIMA(y_train, order=order).fit()
        elif self.model_type == 'SARIMA':
            self.model = SARIMAX(y_train, order=order, seasonal_order=seasonal_order).fit()
        # Guardar la información del modelo ajustado
        self.model_info = {
            'order': order,
            'seasonal_order': seasonal_order,
            'aic': self.model.aic,
            'bic': self.model.bic
        }
    def predict(self, start, end):
        # Realizar predicciones
        y_pred = self.model.predict(start=start, end=end)
        # Invertir escalado si se proporcionó
        if self.scaler:
            y_pred = self.scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
        return y_pred
# Clase TimeSeriesPipeline ya definida aquí...
def build_pipeline(hp, y_train, model_type='KRRBF', seasonal_order=(1, 1, 1, 4), X=X):
    # Estándarización
    scaler = MinMaxScaler()

    # Definición del modelo basado en el tipo
    if model_type == "KRRBF":
        model = KernelRidge(
            kernel='rbf',
            gamma=hp.Float("gamma", 1e-3, 100, default=1, sampling="log"),
            alpha=hp.Float("alpha", 1e-3, 100, default=1, sampling="log")
        )
    elif model_type == "KNN":
        model = KNeighborsRegressor(
            n_neighbors=hp.Int("n_neighbors", 1, 9, default=3)
        )
    elif model_type == "GP":
        model = GaussianProcessRegressor(
            alpha=hp.Float("alpha", 1e-5, 1e2, default=1, sampling="log"),
            kernel=RBF(length_scale=np.ones(X.shape[1]),length_scale_bounds=(1,1000))#WhiteKernel()+Matern(,nu=0.5),normalize_y=True,
        )
    elif model_type == "Linear":
        model = LinearRegression()
    elif model_type == "Lasso":
        model = Lasso(
            alpha=hp.Float("alpha", 1e-3, 1, default=1, sampling="log")
        )
    elif model_type == "Elastic":
        model = ElasticNet(
            alpha=hp.Float("alpha", 0.1, 1, sampling="log"),
            l1_ratio=hp.Float("l1_ratio", 0.1, 1,sampling="log")
        )
    elif model_type == "SGD":
        model = SGDRegressor(
            alpha=hp.Float("alpha", 1e-3, 1, default=0.01, sampling="log"),
            l1_ratio=hp.Float("l1_ratio", 0, 1, default=0.15)
        )
    elif model_type == "BayesRid":
        model = BayesianRidge(
            alpha_1=hp.Float("alpha_1", 1e-6, 1, default=1e-6, sampling="log"),
            alpha_2=hp.Float("alpha_2", 1e-6, 1, default=1e-6, sampling="log"),
            lambda_1=hp.Float("lambda_1", 1e-6, 1, default=1e-6, sampling="log"),
            lambda_2=hp.Float("lambda_2", 1e-6, 1, default=1e-6, sampling="log")
        )
    elif model_type == "RF":
        model = RandomForestRegressor(
            max_depth=hp.Int("max_depth", 1, 100, step=1, default=30),
            n_estimators=hp.Int("n_estimators", 1, 100, step=1, default=70)
        )
    elif model_type == "SVR":
        model = SVR(
            kernel=hp.Choice("kernel", ['linear', 'poly', 'rbf', 'sigmoid'], default='rbf'),
            C=hp.Float("C", 0.1, 1, default=1),
            epsilon=hp.Float("epsilon", 1e-3, 0.1, default=0.1, sampling="log")
        )
    elif model_type in ['ARIMA', 'SARIMA']:
        # Crear una instancia de TimeSeriesPipeline para ARIMA o SARIMA
        seasonal = (model_type == 'SARIMA')
        ts_pipeline = TimeSeriesPipeline(
            model_type=model_type,
            seasonal=seasonal,
            scaler=scaler,
            m=seasonal_order[3] if seasonal else 1
        )
        # Ajustar el modelo con los datos de entrenamiento
        ts_pipeline.fit(y_train)
        # Retornar el modelo ajustado y la información del modelo
        fitted_model = ts_pipeline
        model_info = ts_pipeline.model_info
        return fitted_model, model_info
    else:
        raise ValueError("Unrecognized model_type")
    # Creación del pipeline para los modelos de ML estándar
    pipeline =Pipeline([("reg", model)]) #Pipeline([('sca',scaler),("reg", model)])
    return pipeline
class ModelBuilder:
    def __init__(self, model_type, y_train, seasonal_order=(1, 1, 1, 3)):
        self.model_type = model_type
        self.seasonal_order = seasonal_order
        self.y_train = y_train
    def build_model(self, hp):
        return build_pipeline(hp, self.y_train, self.model_type, self.seasonal_order)
path='/content/drive/MyDrive/Doctorado/Optimización/'
def time_series_cv(X, y, model_type='KRRBF', s=0, Na='', seasonal_order=(1, 1, 1, 1),train_years = 1,test_months = 3,step_months = 3 ):
    tuners = []
    Cont = 0
    y_true_list = []
    y_pred_list = []
    y_std_list = []
    best_params_list = []
      # Años para entrenamiento
      # Meses para predicción
     # Meses para mover hacia adelante
    for ind in range(12, X.shape[0] - test_months + 1, step_months):
        # Definir los intervalos de entrenamiento y prueba
        X_train, y_train = X[(ind - train_years * 12)*s:ind], y[(ind - train_years * 12)*s:ind]
        X_test, y_test = X[ind:ind + test_months], y[ind:ind + test_months]
        if model_type in ['ARIMA', 'SARIMA']:
            model,best_params = ModelBuilder(model_type, y_train, seasonal_order).build_model(2)
            y_pred = model.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1)
        else:
            model_builder = ModelBuilder(model_type, y_train, seasonal_order)
            tuner = CustomSklearnBayesianOptimization(
                hypermodel=model_builder.build_model,
                objective=Objective('score', direction='max'),
                max_trials=15,
                scoring=make_scorer(r2_score),
                directory='.',
                cv=None,
                project_name=f'12{Na}CV{Cont}_{model_type}'
            )
            print(Cont)
            Cont += 1
            tuner.search(X_train, y_train)
            tuners.append(tuner)
            # Obtener y evaluar el mejor modelo de este fold
            best_model = tuner.get_best_models(num_models=1)[0]
            best_params = tuner.get_best_hyperparameters()[0].values
            if model_type == 'GP':
                kernel_params = best_model.get_params()['reg'].kernel_.get_params()
                best_params.update(kernel_params)
                y_pred, y_std = best_model.predict(X_test, return_std=True)
                y_std_list.extend(y_std)
            elif model_type in  ['Lasso','Elastic']:
                # Extraer coeficientes de ElasticNet
                final_model = best_model.named_steps['reg']
                coef_params = final_model.coef_
                best_params.update({'coef_' + str(i): coef for i, coef in enumerate(coef_params)})
                y_pred = best_model.predict(X_test)
            elif model_type == 'RF':
                final_model = best_model.named_steps['reg']
                importances = final_model.feature_importances_
                best_params.update({'feature_importance_' + str(i): imp for i, imp in enumerate(importances)})
                y_pred = best_model.predict(X_test)
            else:
                y_pred = best_model.predict(X_test)
        best_params_list.append(best_params)
        y_true_list.extend(y_test)
        y_pred_list.extend(y_pred)
    # Guardar y_true, y_pred y y_std (si existe) en un archivo Excel
    results_dict = {'y_true': y_true_list, 'y_pred': y_pred_list}
    if y_std_list:
        results_dict['y_std'] = y_std_list
    y_results = pd.DataFrame(results_dict)
    y_results_file_path = f'{path}{test_months} Meses/Results/{model_type}_{s}.xlsx'
    y_results.to_excel(y_results_file_path, index=False)
    # Guardar los mejores hiperparámetros (incluyendo kernel si aplica) en un archivo Excel
    best_params_df = pd.DataFrame(best_params_list)
    best_params_file_path = f'{path}{test_months} Meses/Params/{model_type}_{s}.xlsx'
    best_params_df.to_excel(best_params_file_path, index=False)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.2 MB/s eta 0:00:00


#Data

In [ ]:
FILEID ="1BidLKcEXF6h-LKKWixtKmQ_g5hgvrkgd"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O datos.xlsx && rm -rf /tmp/cookies.txt
!dir

import pandas as pd
Xdata = pd.read_excel('datos.xlsx')
Xdata.head()
Xdata.fillna(Xdata.mean(), inplace=True)
Xdata.describe()



Xdata=Xdata[['Año', 'Mes', 'TGP',  'salarios reales', 'Indicador de Seguimiento a la Economía',
       'imporenta real', 'iva real',
       'Tasa de intervención de política monetaria (%)','TD']]#'TD-1','TD-2','TD-3' ,'TD']]
Xdata.rename(columns={'Año': 'Year', 'Mes':'Month', 'TGP':'GPR',
       'salarios reales':'RS', 'Indicador de Seguimiento a la Economía':'EMI',
       'imporenta real':'RIT', 'iva real':'RVT',
       'Tasa de intervención de política monetaria (%)':'MPIR','TD':'UR'},inplace=True) #'TD-1':'UR-1','TD-2':'UR-2','TD-3':'UR-3','TD':'UR'}, inplace=True)


X_new=Xdata.loc[1:,['GPR','RS','EMI','RIT','RVT','MPIR','UR']]#,
Xdata=Xdata.iloc[:-1]
Xdata[['GPR-1','RS-1','EMI-1','RIT-1','RVT-1','MPIR-1','UR-1']]=X_new.values#'GPR-1','RS-1','EMI-1','RIT-1','RVT-1','MPIR-1',

#Xdata=pd.DataFrame(scaler.fit_transform(Xdata),columns=Xdata.columns)
X_new=Xdata.loc[1:,['GPR-1','RS-1','EMI-1','RIT-1','RVT-1','MPIR-1','UR-1']]#'GPR','RS','EMI','RIT','RVT','MPIR',
Xdata=Xdata.iloc[:-1]
Xdata[['GPR-2','RS-2','EMI-2','RIT-2','RVT-2','MPIR-2','UR-2']]=X_new.values#'GPR-1','RS-1','EMI-1','RIT-1','RVT-1','MPIR-1',

#Xdata=pd.DataFrame(scaler.fit_transform(Xdata),columns=Xdata.columns)
X_new=Xdata.loc[1:,['GPR-2','RS-2','EMI-2','RIT-2','RVT-2','MPIR-2','UR-2']]#'GPR','RS','EMI','RIT','RVT','MPIR',
Xdata=Xdata.iloc[:-1]
Xdata[['GPR-3','RS-3','EMI-3','RIT-3','RVT-3','MPIR-3','UR-3']]=X_new.values#'GPR-1','RS-1','EMI-1','RIT-1','RVT-1','MPIR-1',

#Xdata=pd.DataFrame(scaler.fit_transform(Xdata),columns=Xdata.columns)
X_new=Xdata.loc[1:,['GPR-3','RS-3','EMI-3','RIT-3','RVT-3','MPIR-3','UR-3']]#'GPR','RS','EMI','RIT','RVT','MPIR',
Xdata=Xdata.iloc[:-1]
Xdata[['GPR-4','RS-4','EMI-4','RIT-4','RVT-4','MPIR-4','UR-4']]=X_new.values#'GPR-1','RS-1','EMI-1','RIT-1','RVT-1','MPIR-1',
#Xdata=pd.DataFrame(scaler.fit_transform(Xdata),columns=Xdata.columns)
X_new=Xdata.loc[1:,['GPR-4','RS-4','EMI-4','RIT-4','RVT-4','MPIR-4','UR-4']]#'GPR','RS','EMI','RIT','RVT','MPIR',
Xdata=Xdata.iloc[:-1]
Xdata[['GPR-5','RS-5','EMI-5','RIT-5','RVT-5','MPIR-5','UR-5']]=X_new.values#'GPR-1','RS-1','EMI-1','RIT-1','RVT-1','MPIR-1',
# X_new=Xdata.loc[1:,'TD-1']
# Xdata=Xdata.iloc[:-1]
# Xdata['TD-2']=X_new.values
# X_new=Xdata.loc[1:,'TD-2']
# Xdata=Xdata.iloc[:-1]
# Xdata['TD-3']=X_new.values
print(Xdata.info())
scaler= MinMaxScaler()
Xdata.iloc[:,2:]=scaler.fit_transform(Xdata.iloc[:,2:])
X=Xdata.drop(['UR-5','Year','Month'], axis=1).values
y=Xdata['UR-5'].values

Xdata
i=['KRRBF','KNN','GP','Linear','Lasso','Elastic','SGD','BayesRid','RF','SVR']
Xdata

# Time series cross validation

##Classic Economic Models

In [ ]:
# Evaluar el modelo ARIMA
results_arima = time_series_cv(X,y, model_type='ARIMA',s=0)

In [ ]:
# Evaluar el modelo SARIMA
results_sarima = time_series_cv(X,y, model_type='SARIMA',s=0,seasonal_order=(3, 3, 3, 2))

##Machine Learning Models

In [ ]:
results_KRRBF=time_series_cv(X, y,model_type=i[0],s=0,Na='s0')

In [ ]:
results_KNN=time_series_cv(X, y,model_type=i[1],s=0,Na='s0')

In [ ]:
results_GP=time_series_cv(X, y,model_type=i[2],s=0,Na='s04')

In [ ]:
results_Linear=time_series_cv(X, y,model_type=i[3],s=0,Na='s01')

In [ ]:
results_Lasso=time_series_cv(X, y,model_type=i[4],s=0,Na='s03')

In [ ]:
##########################
results_Elastic=time_series_cv(X, y,model_type=i[5],s=0,Na='s01')

In [ ]:
results_SGD=time_series_cv(X, y,model_type=i[6],s=0,Na='s0')

In [ ]:
results_BayesRid=time_series_cv(X, y,model_type=i[7],s=0,Na='s0')

In [ ]:
results_RF=time_series_cv(X, y,model_type=i[8],s=0,Na='s01')

In [ ]:
results_SVR=time_series_cv(X, y,model_type=i[9],s=0,Na='s0')

# Sliding Window Cross-Validation

##Classic Economic Models

In [ ]:
# Evaluar el modelo ARIMA
results_arima_s = time_series_cv(X,y, model_type='ARIMA',s=1)

In [ ]:
# Evaluar el modelo SARIMA
results_sarima_s = time_series_cv(X,y, model_type='SARIMA',s=1,seasonal_order=(3, 3, 3, 2))

##Machine Learning Models

In [ ]:
results_KRRBF_s=time_series_cv(X, y,model_type=i[0],s=1,Na='s1')

In [ ]:
results_KNN_s=time_series_cv(X, y,model_type=i[1],s=1,Na='s1')

In [ ]:
results_GP_s=time_series_cv(X, y,model_type=i[2],s=1,Na='s12')

In [ ]:
results_Linear_s=time_series_cv(X, y,model_type=i[3],s=1,Na='s1')

In [ ]:
results_Lasso_s=time_series_cv(X, y,model_type=i[4],s=1,Na='s12')

In [ ]:
results_Elastic_s=time_series_cv(X, y,model_type=i[5],s=1,Na='s19')

In [ ]:
results_SGD_s=time_series_cv(X, y,model_type=i[6],s=1,Na='s1')

In [ ]:
results_BayesRid_s=time_series_cv(X, y,model_type=i[7],s=1,Na='s1')

In [ ]:
results_RF_s=time_series_cv(X, y,model_type=i[8],s=1,Na='s11')

In [ ]:
results_SVR_s=time_series_cv(X, y,model_type=i[9],s=1,Na='s1')